# Détection de spams SMS

Nous allons appliquer un RNN sur une tâche que l'on a essayé de résoudre auparavant avec des modèles statistiques plus basiques, mais qui est depuis maîtrisée avec le Deep Learning : la **détection de spams**.

In [27]:
import os
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker

from sagemaker import get_execution_role  # Permet de récupérer le rôle actuel du Studio

On configure le pipeline que l'on va créer.

In [28]:
sess = boto3.Session()
sm = sess.client("sagemaker")
role = get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

model_package_group_name = "spams-sms"
prefix = "pipeline-spam-sms"
pipeline_name = "pipeline-spam-sms"

On récupère les données.

In [29]:
!mkdir -p data
!mkdir -p data/primary
# !curl -o data/spams.csv https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/deep_learning/data/spam.csv

"""
!curl -o data/raw_data_01_01.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_01_df.csv
!curl -o data/raw_data_01_02.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_02_df.csv
!curl -o data/raw_data_01_03.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_03_df.csv
!curl -o data/raw_data_01_04.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_04_df.csv
!curl -o data/raw_data_01_05.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_05_df.csv
!curl -o data/raw_data_01_06.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_06_df.csv
!curl -o data/raw_data_01_07.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_07_df.csv
!curl -o data/raw_data_01_08.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_08_df.csv
!curl -o data/raw_data_01_09.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_09_df.csv
!curl -o data/raw_data_01_10.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_10_df.csv
!curl -o data/raw_data_01_11.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_11_df.csv
!curl -o data/raw_data_01_12.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_12_df.csv
!curl -o data/raw_data_01_13.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_13_df.csv
!curl -o data/raw_data_01_14.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_14_df.csv
!curl -o data/raw_data_01_15.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_15_df.csv
!curl -o data/raw_data_01_16.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_16_df.csv
!curl -o data/raw_data_01_17.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_17_df.csv
!curl -o data/raw_data_01_18.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_18_df.csv
!curl -o data/raw_data_01_19.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_19_df.csv
"""
!curl -o data/raw_data_01_20.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_20_df.csv
!curl -o data/raw_data_01_21.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_21_df.csv
!curl -o data/raw_data_01_22.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_22_df.csv
!curl -o data/raw_data_01_23.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_23_df.csv
!curl -o data/raw_data_01_24.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_24_df.csv
!curl -o data/raw_data_01_25.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_25_df.csv
!curl -o data/raw_data_01_26.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_26_df.csv
!curl -o data/raw_data_01_27.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_27_df.csv
!curl -o data/raw_data_01_28.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_28_df.csv
!curl -o data/raw_data_01_29.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_29_df.csv
!curl -o data/raw_data_01_30.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_30_df.csv
!curl -o data/raw_data_01_31.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_01_31_df.csv
!curl -o data/raw_data_02_01.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_01_df.csv
!curl -o data/raw_data_02_02.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_02_df.csv
!curl -o data/raw_data_02_03.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_03_df.csv
!curl -o data/raw_data_02_04.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_04_df.csv
!curl -o data/raw_data_02_05.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_05_df.csv
!curl -o data/raw_data_02_06.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_06_df.csv
!curl -o data/raw_data_02_07.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_07_df.csv
!curl -o data/raw_data_02_08.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_08_df.csv
!curl -o data/raw_data_02_09.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_09_df.csv
!curl -o data/raw_data_02_10.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_10_df.csv
!curl -o data/raw_data_02_11.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_11_df.csv
!curl -o data/raw_data_02_12.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_12_df.csv
!curl -o data/raw_data_02_13.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_13_df.csv
!curl -o data/raw_data_02_14.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_14_df.csv
!curl -o data/raw_data_02_15.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_15_df.csv
!curl -o data/raw_data_02_16.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_16_df.csv
!curl -o data/raw_data_02_17.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_17_df.csv
!curl -o data/raw_data_02_18.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_18_df.csv
!curl -o data/raw_data_02_19.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_19_df.csv
!curl -o data/raw_data_02_20.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_20_df.csv
!curl -o data/raw_data_02_21.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_21_df.csv
!curl -o data/raw_data_02_22.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_22_df.csv
!curl -o data/raw_data_02_23.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_23_df.csv
!curl -o data/raw_data_02_24.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_24_df.csv
!curl -o data/raw_data_02_25.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_25_df.csv
!curl -o data/raw_data_02_26.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_26_df.csv
!curl -o data/raw_data_02_27.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_27_df.csv
!curl -o data/raw_data_02_28.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_28_df.csv
!curl -o data/raw_data_02_29.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_29_df.csv
!curl -o data/raw_data_02_30.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_30_df.csv
!curl -o data/raw_data_02_31.csv https://esginyprojectdailyml.s3.eu-west-3.amazonaws.com/2019_02_31_df.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34797  100 34797    0     0  1029k      0 --:--:-- --:--:-- --:--:-- 1029k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.7M  100 17.7M    0     0  57.4M      0 --:--:-- --:--:-- --:--:-- 57.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.4M  100 23.4M    0     0  91.0M      0 --:--:-- --:--:-- --:--:-- 91.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.0M  100 24.0M    0     0  87.0M      0 --:--:-- --:--:-- --:--:-- 86.7M
  % Total    % Received % Xferd  Average Speed   Tim

In [30]:

full_df = None
for day in range(1,32):
    for month in range(1,3):
        fname = f"data/raw_data_{month:02d}_{day:02d}.csv"
        print(fname)
        try:
            df = pd.read_csv(fname)
            full_df = pd.concat([full_df,df])
        except:
            pass

            
full_df.to_csv(f"./data/primary.csv", header=True, index=False)
rawdata_s3_prefix = "{}/data/primary".format(prefix)
raw_s3 = sagemaker_session.upload_data(path="./data/", key_prefix=rawdata_s3_prefix)
print(raw_s3)
"""
spams_df = pd.read_csv("data/spams.csv")
spams_df.to_csv(f"./data/primary.csv", header=True, index=False)
rawdata_s3_prefix = "{}/data/primary".format(prefix)
raw_s3 = sagemaker_session.upload_data(path="./data/", key_prefix=rawdata_s3_prefix)
print(raw_s3)
"""

data/raw_data_01_01.csv
data/raw_data_02_01.csv
data/raw_data_01_02.csv
data/raw_data_02_02.csv
data/raw_data_01_03.csv
data/raw_data_02_03.csv
data/raw_data_01_04.csv
data/raw_data_02_04.csv
data/raw_data_01_05.csv
data/raw_data_02_05.csv
data/raw_data_01_06.csv
data/raw_data_02_06.csv
data/raw_data_01_07.csv
data/raw_data_02_07.csv
data/raw_data_01_08.csv
data/raw_data_02_08.csv
data/raw_data_01_09.csv
data/raw_data_02_09.csv
data/raw_data_01_10.csv
data/raw_data_02_10.csv
data/raw_data_01_11.csv
data/raw_data_02_11.csv
data/raw_data_01_12.csv
data/raw_data_02_12.csv
data/raw_data_01_13.csv
data/raw_data_02_13.csv
data/raw_data_01_14.csv
data/raw_data_02_14.csv
data/raw_data_01_15.csv
data/raw_data_02_15.csv
data/raw_data_01_16.csv
data/raw_data_02_16.csv
data/raw_data_01_17.csv
data/raw_data_02_17.csv
data/raw_data_01_18.csv
data/raw_data_02_18.csv
data/raw_data_01_19.csv
data/raw_data_02_19.csv
data/raw_data_01_20.csv
data/raw_data_02_20.csv
data/raw_data_01_21.csv
data/raw_data_02

'\nspams_df = pd.read_csv("data/spams.csv")\nspams_df.to_csv(f"./data/primary.csv", header=True, index=False)\nrawdata_s3_prefix = "{}/data/primary".format(prefix)\nraw_s3 = sagemaker_session.upload_data(path="./data/", key_prefix=rawdata_s3_prefix)\nprint(raw_s3)\n'

In [31]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="1D")

## Pipeline - Processing

In [32]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat

input_data = ParameterString(name="InputData", default_value=raw_s3)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.large"
)

In [33]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=sklearn_framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-preprocess-sms",
    role=role,
)

In [34]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

step_process = ProcessingStep(
    name="Preprocess",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="tokenizer", source="/opt/ml/processing/tokenizer"),
        ProcessingOutput(output_name="datasets", source="/opt/ml/processing/datasets")
    ],
    code="codes/preprocess.py",
    cache_config=cache_config
)

In [35]:
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        input_data
    ],
    steps=[step_process],
)

In [36]:
#pipeline.create(role_arn=role)
#pipeline.start()

In [37]:
#pipeline.delete()

## Pipeline - Training

In [38]:
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")

In [39]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
import time

# Chemin d'output du modèle
model_path = f"s3://{bucket}/{prefix}/model/"

hyperparameters = {}
tensorflow_version = "2.4.1"
python_version = "py37"

tf_estimator = TensorFlow(
    source_dir="codes",
    entry_point="train.py",
    instance_type=training_instance_type,
    instance_count=1,
    framework_version=tensorflow_version,
    role=role,
    base_job_name="tensorflow-train-model",
    output_path=model_path,
    hyperparameters=hyperparameters,
    py_version=python_version,
)

step_train = TrainingStep(
    name="Train",
    estimator=tf_estimator,
    inputs={
        "datasets": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "datasets"
            ].S3Output.S3Uri,
            content_type="text/csv",
        )
    },
    cache_config=cache_config
)

In [40]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        input_data
    ],
    steps=[step_process, step_train],
)

In [41]:
#pipeline.create(role_arn=role)
#pipeline.start()

In [42]:
#pipeline.delete()

## Pipeline - Evaluate

In [43]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.sklearn.processing import ScriptProcessor

tf_eval_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version=tensorflow_version,
    image_scope="training",
    py_version="py37",
    instance_type=training_instance_type,
)

evaluate_model_processor = ScriptProcessor(
    role=role, 
    image_uri=tf_eval_image_uri, 
    command=['python3'], 
    instance_count=1, 
    instance_type=training_instance_type, 
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

# Use the evaluate_model_processor in a Sagemaker pipelines ProcessingStep.
step_evaluate = ProcessingStep(
    name="Evaluate",
    processor=evaluate_model_processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "datasets"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/datasets",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="codes/evaluate.py",
    property_files=[evaluation_report],
)

In [44]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        input_data
    ],
    steps=[step_process, step_train, step_evaluate],
)

In [45]:
#pipeline.create(role_arn=role)
#pipeline.start()

In [46]:
#pipeline.delete()

## Pipeline - Model Registry

In [54]:
from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import PipelineModel

tokenizer_s3 = "{}/tokenizer.tar.gz".format(
    step_process.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]  # Sortie Tokenizer
)

tokenizer_model = SKLearnModel(
    model_data=tokenizer_s3,
    role=role,
    sagemaker_session=sagemaker_session,
    entry_point="codes/preprocess.py",
    framework_version=sklearn_framework_version
)

tf_model_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version=tensorflow_version,
    image_scope="inference",
    py_version="py37",
    instance_type=training_instance_type,
)

tf_model = Model(
    image_uri=tf_model_image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

pipeline_model = PipelineModel(
    models=[tokenizer_model, tf_model],
    role=role,
    sagemaker_session=sagemaker_session
)

In [55]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="Approved")

evaluation_s3_uri = "{}/evaluation.json".format(
    step_evaluate.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

step_register_pipeline_model = RegisterModel(
    name="RegisterPipelineModel",
    model=pipeline_model,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    cache_config=cache_config,
    model_metrics=model_metrics
)

In [56]:
"""
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

accuracy_threshold = ParameterFloat(name="AccuracyThreshold", default_value=0.8)

# Create accuracy condition to ensure the model meets performance requirements.
# Models with a test accuracy lower than the condition will not be registered with the model registry.
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=step_evaluate,
        property_file=evaluation_report,
        json_path="classification_metrics.accuracy.value",
    ),
    right=accuracy_threshold,
)

# Create a Sagemaker Pipelines ConditionStep, using the condition above.
# Enter the steps to perform if the condition returns True / False.
step_cond = ConditionStep(
    name="AccOverThreshold",
    conditions=[cond_gte],
    if_steps=[step_register_pipeline_model],
    else_steps=[],
)
"""

'\nfrom sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo\nfrom sagemaker.workflow.condition_step import (\n    ConditionStep,\n    JsonGet,\n)\n\naccuracy_threshold = ParameterFloat(name="AccuracyThreshold", default_value=0.8)\n\n# Create accuracy condition to ensure the model meets performance requirements.\n# Models with a test accuracy lower than the condition will not be registered with the model registry.\ncond_gte = ConditionGreaterThanOrEqualTo(\n    left=JsonGet(\n        step=step_evaluate,\n        property_file=evaluation_report,\n        json_path="classification_metrics.accuracy.value",\n    ),\n    right=accuracy_threshold,\n)\n\n# Create a Sagemaker Pipelines ConditionStep, using the condition above.\n# Enter the steps to perform if the condition returns True / False.\nstep_cond = ConditionStep(\n    name="AccOverThreshold",\n    conditions=[cond_gte],\n    if_steps=[step_register_pipeline_model],\n    else_steps=[],\n)\n'

In [57]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        input_data,
        model_approval_status,
        #accuracy_threshold
    ],
    #steps=[step_process, step_train, step_evaluate, step_cond],
    steps=[step_process, step_train, step_evaluate],
)

In [59]:
pipeline.create(role_arn=role)
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:eu-west-3:167304041891:pipeline/pipeline-spam-sms/execution/rvwy1pyvnl7k', sagemaker_session=<sagemaker.session.Session object at 0x7fcad7e3ba50>)

In [58]:
pipeline.delete()


{'PipelineArn': 'arn:aws:sagemaker:eu-west-3:167304041891:pipeline/pipeline-spam-sms',
 'ResponseMetadata': {'RequestId': '203c22ae-4b4b-43ac-800a-8a646e2b5766',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '203c22ae-4b4b-43ac-800a-8a646e2b5766',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'date': 'Thu, 20 Jan 2022 12:47:03 GMT'},
  'RetryAttempts': 0}}

## Pipeline - Deploy

In [64]:
from codes.utils import get_approved_package

sm_client = boto3.client("sagemaker")

print(model_package_group_name)
pck = get_approved_package(model_package_group_name)
model_description = sm_client.describe_model_package(ModelPackageName=pck['ModelPackageArn'])
model_description

No approved ModelPackage found for ModelPackageGroup: spams-sms


spams-sms


Exception: No approved ModelPackage found for ModelPackageGroup: spams-sms

In [ ]:
from sagemaker import ModelPackage

model_package_arn = model_description['ModelPackageArn']
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session
)

endpoint_name = 'spams-sms-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("EndpointName= {}".format(endpoint_name))
model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge', endpoint_name=endpoint_name)

## Création d'un prédicteur

In [ ]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name=endpoint_name)

In [ ]:
import json

p = predictor.predict(
    json.dumps({"samples": ["I am going to the park.", "Call from 08702490080 - tells u 2 call 09066358152 to claim �5000 prize."]}),
    initial_args = {"ContentType": "application/json"}
)

json.loads(p)

In [ ]:
!cd
!tar -czf archive.tar.gz .

In [ ]:
!ls -halt


In [ ]:
!ls /opt/ml/processing/